In [1]:
# Importing dependencies
import pandas as pd 
from sqlalchemy import create_engine

In [2]:
# Establishing SQL connection
rds_connection_string = "postgres:Tenors2015@localhost:5432/education_taxes_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
# Importing tax data.
csv = 'tax_data.csv'
dirty_taxes = pd.read_csv(csv)
dirty_taxes.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,ELF,CPREP,PREP,DIR_DEP,N2,NUMDEP,TOTAL_VITA,VITA,TCE,VITA_EIC
0,1,AL,0,1,768120,696930,37470,399160,559820,1180240,439980,24700,16610,8100,5300
1,1,AL,0,2,503430,457510,23180,266880,371440,977450,348420,12230,8350,3880,350
2,1,AL,0,3,274590,248630,13210,157800,179490,587740,182370,3110,1590,1520,0
3,1,AL,0,4,174830,159190,6830,102340,110320,429360,131170,990,510,480,0
4,1,AL,0,5,245150,224280,10500,145010,123560,665630,213100,770,730,40,0


In [4]:
# Filtering the data to show only IN values & dropping unneeded columns.
ind_tax=dirty_taxes.loc[dirty_taxes["STATE"]=='IN']
ind_tax = ind_tax.drop(['STATEFIPS', 'STATE', 'VITA', 'TCE', 'VITA_EIC'], axis=1)
ind_tax.reset_index(drop=True, inplace=True)
ind_tax.head()

,zipcode,agi_stub,N1,ELF,CPREP,PREP,DIR_DEP,N2,NUMDEP,TOTAL_VITA
0,0,1,1086910,984770,54800,458270,737010,1393890,403520,38090
1,0,2,777250,720010,29210,355760,595250,1416310,465680,13090
2,0,3,451080,416800,16870,232700,318230,971370,305680,2850
3,0,4,298350,275970,10310,162530,204370,750610,236570,210
4,0,5,396630,367030,15440,223710,224300,1108230,371700,0


In [5]:
# Renaming columns.
clean_tax=ind_tax.rename(columns={'zipcode':'ZIP', 'agi_stub':'AGI', 'N1':'Total_Returns', 'ELF':'Electronically_Filed',
                                 'CPREP':'Computer_Prepared_Paper', 'PREP':'Paid_Preparer', 'DIR_DEP':'Direct_Deposit',
                                 'N2':'Total_Persons', 'NUMDEP':'Total_Dependents', 'TOTAL_VITA':'Volunteer_Prepared'})
clean_tax.head()

,ZIP,AGI,Total_Returns,Electronically_Filed,Computer_Prepared_Paper,Paid_Preparer,Direct_Deposit,Total_Persons,Total_Dependents,Volunteer_Prepared
0,0,1,1086910,984770,54800,458270,737010,1393890,403520,38090
1,0,2,777250,720010,29210,355760,595250,1416310,465680,13090
2,0,3,451080,416800,16870,232700,318230,971370,305680,2850
3,0,4,298350,275970,10310,162530,204370,750610,236570,210
4,0,5,396630,367030,15440,223710,224300,1108230,371700,0


In [6]:
# Dropping rows with erronious placehoder data.
clean_tax=clean_tax[clean_tax.ZIP !=99999]
clean_tax.head()

,ZIP,AGI,Total_Returns,Electronically_Filed,Computer_Prepared_Paper,Paid_Preparer,Direct_Deposit,Total_Persons,Total_Dependents,Volunteer_Prepared
0,0,1,1086910,984770,54800,458270,737010,1393890,403520,38090
1,0,2,777250,720010,29210,355760,595250,1416310,465680,13090
2,0,3,451080,416800,16870,232700,318230,971370,305680,2850
3,0,4,298350,275970,10310,162530,204370,750610,236570,210
4,0,5,396630,367030,15440,223710,224300,1108230,371700,0


# Tax Data Normalization

In [7]:
# Normalizing and uploading to PGAdmin
total_returns = clean_tax[['ZIP', 'AGI', 'Total_Returns']]
total_returns.to_sql(name='total_returns', con=engine, if_exists="append", index=False)
total_returns

,ZIP,AGI,Total_Returns
0,0,1,1086910
1,0,2,777250
2,0,3,451080
3,0,4,298350
4,0,5,396630
...,...,...,...
4045,47995,2,190
4046,47995,3,140
4047,47995,4,80
4048,47995,5,90


In [8]:
# Normalizing and uploading to PGAdmin
elf = clean_tax[['ZIP', 'AGI', 'Electronically_Filed']]
elf.to_sql(name='electronically_filed', con=engine, if_exists="append", index=False)
elf

,ZIP,AGI,Electronically_Filed
0,0,1,984770
1,0,2,720010
2,0,3,416800
3,0,4,275970
4,0,5,367030
...,...,...,...
4045,47995,2,180
4046,47995,3,130
4047,47995,4,80
4048,47995,5,90


In [9]:
# Normalizing and uploading to PGAdmin
cpp = clean_tax[['ZIP', 'AGI', 'Computer_Prepared_Paper']]
cpp.to_sql(name='computer_prepared_paper', con=engine, if_exists="append", index=False)
cpp

,ZIP,AGI,Computer_Prepared_Paper
0,0,1,54800
1,0,2,29210
2,0,3,16870
3,0,4,10310
4,0,5,15440
...,...,...,...
4045,47995,2,0
4046,47995,3,0
4047,47995,4,0
4048,47995,5,0


In [10]:
# Normalizing and uploading to PGAdmin
pp = clean_tax[['ZIP', 'AGI', 'Paid_Preparer']]
pp.to_sql(name='paid_preparer', con=engine, if_exists="append", index=False)
pp

,ZIP,AGI,Paid_Preparer
0,0,1,458270
1,0,2,355760
2,0,3,232700
3,0,4,162530
4,0,5,223710
...,...,...,...
4045,47995,2,100
4046,47995,3,80
4047,47995,4,60
4048,47995,5,70


In [11]:
# Normalizing and uploading to PGAdmin
dd = clean_tax[['ZIP', 'AGI', 'Direct_Deposit']]
dd.to_sql(name='direct_deposit', con=engine, if_exists="append", index=False)
dd

,ZIP,AGI,Direct_Deposit
0,0,1,737010
1,0,2,595250
2,0,3,318230
3,0,4,204370
4,0,5,224300
...,...,...,...
4045,47995,2,130
4046,47995,3,100
4047,47995,4,50
4048,47995,5,60


In [12]:
# Normalizing and uploading to PGAdmin
tp = clean_tax[['ZIP', 'AGI', 'Total_Persons']]
tp.to_sql(name='total_persons', con=engine, if_exists="append", index=False)
tp

,ZIP,AGI,Total_Persons
0,0,1,1393890
1,0,2,1416310
2,0,3,971370
3,0,4,750610
4,0,5,1108230
...,...,...,...
4045,47995,2,350
4046,47995,3,320
4047,47995,4,230
4048,47995,5,280


In [13]:
# Normalizing and uploading to PGAdmin
td = clean_tax[['ZIP', 'AGI', 'Total_Dependents']]
td.to_sql(name='total_dependents', con=engine, if_exists="append", index=False)
td

,ZIP,AGI,Total_Dependents
0,0,1,403520
1,0,2,465680
2,0,3,305680
3,0,4,236570
4,0,5,371700
...,...,...,...
4045,47995,2,110
4046,47995,3,110
4047,47995,4,80
4048,47995,5,120


In [14]:
# Normalizing and uploading to PGAdmin
vp = clean_tax[['ZIP', 'AGI', 'Volunteer_Prepared']]
vp.to_sql(name='volunteer_prepared', con=engine, if_exists="append", index=False)
vp

,ZIP,AGI,Volunteer_Prepared
0,0,1,38090
1,0,2,13090
2,0,3,2850
3,0,4,210
4,0,5,0
...,...,...,...
4045,47995,2,0
4046,47995,3,0
4047,47995,4,0
4048,47995,5,0


# K12 Data Normalization

In [12]:
# Importing clean K12 csv data
k12 = 'Indiana_k12.csv'
k12_data = pd.read_csv(k12)
k12_data

,Unnamed: 0,NAME,ADDRESS,CITY,STATE,ZIP,LEVEL,ENROLLMENT,START_GRADE,END_GRADE,DISTRICT_ID
0,0,LOGANSPORT JUVENILE COR FAC,LOGANSPORT ST HOSP-WEST SIDE,LOGANSPORT,IN,46947,3,122,07,12,1800006
1,1,NORTHEASTERN ELEMENTARY SCH,534 W WALLACE RD,FOUNTAIN CITY,IN,47341,1,551,PK,05,1808190
2,2,HAZEL DELL ELEMENTARY SCHOOL,3025 WESTFIELD RD,NOBLESVILLE,IN,46062,1,702,PK,05,1807650
3,3,BROWNSBURG WEST MIDDLE SCHOOL,1555 S ODELL ST,BROWNSBURG,IN,46112,2,914,06,08,1801020
4,4,FRANKLIN ELEMENTARY SCHOOL,410 W MIAMI AVE,LOGANSPORT,IN,46947,1,398,KG,05,1806030
...,...,...,...,...,...,...,...,...,...,...,...
1868,1868,BEN DAVIS UNIVERSITY HIGH SCHOOL,1155 S HIGH SCHOOL RD,INDIANAPOLIS,IN,46241,3,358,10,12,1812810
1869,1869,MADISON CONSOLIDATED JR HIGH SCH,701 8TH ST,MADISON,IN,47250,2,654,06,08,1806120
1870,1870,HAUBSTADT COMMUNITY SCHOOL,158 E 1025 S,HAUBSTADT,IN,47639,1,340,KG,08,1810350
1871,1871,THORPE CREEK ELEMENTARY,14642 E 126TH ST,FISHERS,IN,46038,1,893,KG,04,1810650


In [27]:
# Normalizing and uploading to PGAdmin
enrollment_count = k12_data[['ZIP', 'ENROLLMENT']]
enrollment_count = enrollment_count.groupby(['ZIP']).sum()
enrollment_count = enrollment_count.drop([47989], axis=0)
enrollment_count.to_sql(name='k12_enrollment_count', con=engine, if_exists="append", index=False)
enrollment_count

,ENROLLMENT
ZIP,
46001,1582
46011,942
46012,2761
46013,2721
46016,1202
...,...
47978,1693
47987,1121
47991,862


In [44]:
# Normalizing and uploading to PGAdmin
school_count = k12_data[['ZIP']]
school_count = school_count.value_counts()
school_count = pd.DataFrame(school_count)
school_count = school_count.rename(columns = {0 : "Count"})
school_count.to_sql(name='k12_schools_zip_count', con=engine, if_exists="append", index=False)
school_count

,Count
ZIP,
46224,17
46383,15
47201,14
46901,13
46123,13
...,...
46348,1
47351,1
46346,1


# Census Data Normalization

In [51]:
# Importing and cleaning census csv data
census = 'census_data_indiana_2014.csv'
census_csv = pd.read_csv(census)
census_csv = census_csv.rename(columns = {'Zipcode':'ZIP', 'Household Income':'Household_Income', 
                                          'Per Capita Income':'Per_Capita_Income', 'Poverty Rate':'Poverty_Rate'})
census_csv

,ZIP,Population,Median Age,Household_Income,Per_Capita_Income,Poverty Count,Poverty_Rate
0,46511,4254.0,46.9,52570.0,32929.0,348.0,8.180536
1,46526,32513.0,36.0,54187.0,24312.0,4495.0,13.825239
2,46528,27165.0,32.6,63537.0,26499.0,2439.0,8.978465
3,46544,31359.0,38.0,47778.0,25268.0,4324.0,13.788705
4,46553,3212.0,36.7,62829.0,24039.0,78.0,2.428394
...,...,...,...,...,...,...,...
770,47166,1540.0,44.1,44159.0,24089.0,308.0,20.000000
771,47224,642.0,37.4,45136.0,15881.0,162.0,25.233645
772,47244,737.0,39.3,49659.0,23775.0,157.0,21.302578
773,47274,30970.0,38.1,49644.0,24801.0,4684.0,15.124314


In [50]:
# Normalizing and uploading to PGAdmin
census_pop = census_csv[['ZIP', 'Population']]
census_pop.to_sql(name='census_population', con=engine, if_exists="append", index=False)
census_pop

,ZIP,Population
0,46511,4254.0
1,46526,32513.0
2,46528,27165.0
3,46544,31359.0
4,46553,3212.0
...,...,...
770,47166,1540.0
771,47224,642.0
772,47244,737.0
773,47274,30970.0


In [ ]:
# Normalizing and uploading to PGAdmin
census_hincome = census_csv[['ZIP', 'Household_Income']]
census_hincome.to_sql(name='census_household_income', con=engine, if_exists="append", index=False)
census_hincome

In [ ]:
# Normalizing and uploading to PGAdmin
census_pincome = census_csv[['ZIP', 'Per_Capita_Income']]
census_pincome.to_sql(name='census_per_capita_income', con=engine, if_exists="append", index=False)
census_pincome

In [ ]:
# Normalizing and uploading to PGAdmin
census_poverty = census_csv[['ZIP', 'Poverty_Rate']]
census_poverty.to_sql(name='census_poverty_rate', con=engine, if_exists="append", index=False)
census_poverty

# College Data Normalization

In [53]:
# Importing college csv data
college = 'colleges_unis_with_zips.csv'
college_df = pd.read_csv(college)
college_df.head()

,School,City,State,Lat,Lng,place_id,Zip Code
0,ANCILLA COLLEGE,DONALDSON,IN,41.364484,-86.444133,ChIJNykO2EVaEYgR6iuHCTAnQGc,46563
1,ANDERSON UNIVERSITY,ANDERSON,IN,40.105320,-85.680254,ChIJvRvrHQzZFIgRaG89vy-ipXg,46012
2,BALL STATE UNIVERSITY,MUNCIE,IN,40.193377,-85.386360,ChIJIR58aWY9FYgR9ImfGJvu4OQ,47306
3,BETHEL COLLEGE,MISHAWAKA,IN,41.661993,-86.158616,ChIJf2dC58HNFogRb1W460Xop8g,46545
4,BUTLER UNIVERSITY,INDIANAPOLIS,IN,39.768403,-86.158068,ChIJr8OliPpTa4gRPkUtyy7TxQM,46208


In [54]:
# Normalizing and uploading to PGAdmin
schools_by_zip = college_df["Zip Code"].value_counts()
schools_by_zip_df = pd.DataFrame(schools_by_zip)
schools_by_zip_df.to_sql(name='college_count', con=engine, if_exists="append", index=False)
schools_by_zip_df

,Zip Code
46805,4
46556,3
46208,2
47374,2
47710,2
46590,2
46989,1
46307,1
46545,1
46803,1
